<a href="https://colab.research.google.com/github/ycheung8/de_portfolio/blob/main/notebook/riteaid_covid_vaccine_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
from datetime import datetime
import smtplib
#import math math.isNaN() 

In [ ]:
def initial_ra_file():
    df = pd.read_csv(r'C:\Users\ycheung\Documents\Python_Script\rite_aid\ra_pa_data_1.csv', dtype=str)
    ra_output_name = ['Data','Status','current_dtm','dtm','pos_change','old_slot']
    df2 = pd.concat([df,pd.DataFrame(columns=ra_output_name)], sort=False)
    return df2

In [1]:
#df2 = initial_ra_file()
df2 = pd.read_csv(r'C:\Users\ycheung\Documents\Python_Script\rite_aid\ra_pa_data.csv', dtype=str)

df2['old_slot'] = df2['Data']
df2['pos_change'] = pd.to_numeric(df2['pos_change'])
df2['pos_change'] = 0 

copy_address = []

df2['old_slot'] = df2['Data']

NameError: ignored

In [ ]:
sms_text = ''

#set up email text variables
server = smtplib.SMTP( "smtp.gmail.com", 587 )
server.starttls()
server.login( 'yic.alerts@gmail.com', 'Upgrade135' )
from_mail = 'yic.alerts@gmail.com'

In [ ]:
for store in df2['store_number']:
    # current time
    current_dtm = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    df2.loc[df2.store_number == store,'current_dtm'] = current_dtm

    old_slot = df2.loc[df2['store_number']==store,'old_slot'].values[0] # this stays a string

    # access website
    page_resp = requests.get("https://www.riteaid.com/services/ext/v2/vaccine/checkSlots?storeNumber="+store)
    page = requests.get("https://www.riteaid.com/services/ext/v2/vaccine/checkSlots?storeNumber="+store).text

    df_json = pd.read_json(page)
    slot = (df_json['Data'][0]['1'])
    df2.loc[df2.store_number == store,'Data'] = slot
    df2.loc[df2.store_number == store,'Status'] = df_json["Status"][0]

    #values to change pos_change field, which generates alert
    if (slot == True and old_slot == 'False'):
        df2.loc[df2.store_number == store,'pos_change'] = 1
        df2.loc[df2.store_number == store,'dtm'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        copy_address.append(str(df2.loc[df2.store_number == store,'address_1'].iloc[0] + ' ' + df2.loc[df2.store_number == store,'city'].iloc[0] + ' ' + df2.loc[df2.store_number == store,'state'].iloc[0] + ' ' + df2.loc[df2.store_number == store,'zip'].iloc[0] + ' '))
        copy_address_1 = str(df2.loc[df2.store_number == store,'address_1'].iloc[0] + ' ' + df2.loc[df2.store_number == store,'city'].iloc[0] + ' ' + df2.loc[df2.store_number == store,'state'].iloc[0] + ' ' + df2.loc[df2.store_number == store,'zip'].iloc[0] + ' ')            

        message = ("From: %s\r\n" % from_mail + "To: %s\r\n" % from_mail + "Subject: %s\r\n" % '' + "\r\n" + sms_text.join(copy_address_1))
        server.sendmail(from_mail, from_mail, message)
            
    else:
        df2.loc[df2.store_number == store,'pos_change'] = 0

In [ ]:
server.quit()  

In [ ]:
df2 = df2[['store_number','address_1','city','state','zip','Data','Status','current_dtm','dtm','pos_change','old_slot']]
#print(df2)
df2.to_csv("C:/Users/ycheung/Documents/Python_Script/rite_aid/ra_pa_data"+"_"+datetime.now().strftime('%H_%M')+".csv", encoding='utf-8')
df2.to_csv("C:/Users/ycheung/Documents/Python_Script/rite_aid/ra_pa_data.csv", encoding='utf-8')